In [1]:
import torch
import torch.fx

In [2]:
class MyModule(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.param = torch.nn.Parameter(torch.rand(3, 4))
        self.linear = torch.nn.Linear(4, 5)

    def forward(self, x):
        w = self.linear.weight
        x = x + w
        x = self.linear(x)
        x = x.relu()
        x = torch.sum(x ,dim=-1)
        x = torch.topk(x, 3)
        return x

In [4]:
m = MyModule()
gm = torch.fx.symbolic_trace(m)
print(gm.graph)
print(gm.code)

graph():
    %x : [num_users=1] = placeholder[target=x]
    %linear_weight : [num_users=1] = get_attr[target=linear.weight]
    %add : [num_users=1] = call_function[target=operator.add](args = (%x, %linear_weight), kwargs = {})
    %linear : [num_users=1] = call_module[target=linear](args = (%add,), kwargs = {})
    %relu : [num_users=1] = call_method[target=relu](args = (%linear,), kwargs = {})
    %sum_1 : [num_users=1] = call_function[target=torch.sum](args = (%relu,), kwargs = {dim: -1})
    %topk : [num_users=1] = call_function[target=torch.topk](args = (%sum_1, 3), kwargs = {})
    return topk



def forward(self, x):
    linear_weight = self.linear.weight
    add = x + linear_weight;  x = linear_weight = None
    linear = self.linear(add);  add = None
    relu = linear.relu();  linear = None
    sum_1 = torch.sum(relu, dim = -1);  relu = None
    topk = torch.topk(sum_1, 3);  sum_1 = None
    return topk
    


In [6]:
list(gm.graph.nodes)

[x, linear_weight, add, linear, relu, sum_1, topk, output]